# Nuclio - Generator function

## Environment

In [1]:
# nuclio: ignore
import nuclio

### Configurations

In [2]:
%%nuclio config 

# Trigger
spec.triggers.secs.kind = "cron"
spec.triggers.secs.attributes.interval = "10s"

# Base image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.secs.kind to 'cron'
%nuclio: setting spec.triggers.secs.attributes.interval to '10s'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Mount

In [15]:
os.environ['configurations_path'] = os.path.join(os.getcwd(), 'configurations')

In [16]:
%nuclio mount /configurations ${configurations_path}

mounting volume path /configurations as /User/netops/tutorials/demos/netops/configurations


### Setups

In [4]:
%%nuclio cmd

pip install pytimeparse
pip install -i https://test.pypi.org/simple/ v3io-generator
pip install faker
pip install pyarrow

mkdir /configurations

Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
Looking in indexes: https://test.pypi.org/simple/
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'
    100% |################################| 880kB 26.3MB/s 
    100% |################################| 81kB 20.6MB/s 
Invalid -W option ignored: invalid action: 'yaml.YAMLLoadWarning'


### Variables

In [22]:
%%nuclio env

# Deployment
SAVE_DEPLOYMENT=True
DEPLOYMENT_TABLE=netops_devices

# Metrics
SAVE_TO=netops_metrics
INITIAL_TIMESTAMP=-1
SECS_TO_GENERATE=10

# Save as
SAVE_TO_TSDB=Truee

%nuclio: setting 'SAVE_DEPLOYMENT' environment variable
%nuclio: setting 'DEPLOYMENT_TABLE' environment variable
%nuclio: setting 'SAVE_TO' environment variable
%nuclio: setting 'INITIAL_TIMESTAMP' environment variable
%nuclio: setting 'SECS_TO_GENERATE' environment variable
%nuclio: setting 'SAVE_TO_TSDB' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


In [54]:
%nuclio env -c METRICS_CONFIGURATION_FILEPATH=/configurations/metrics_configuration.yaml
%nuclio env -l METRICS_CONFIGURATION_FILEPATH=configurations/metrics_configuration.yaml

%nuclio: setting 'METRICS_CONFIGURATION_FILEPATH' environment variable


## Function

In [55]:
import os
import time
import yaml
import pandas as pd
import datetime
import itertools

# DB Connection
import v3io_frames as v3f

# Data generator
from v3io_generator import metrics_generator, deployment_generator

### Helper functions

In [56]:
def _create_deployment():
    print('creating deployment')
    # Create meta-data factory
    dep_gen = deployment_generator.deployment_generator()
    faker=dep_gen.get_faker()

    # Design meta-data
    dep_gen.add_level(name='company',number=2,level_type=faker.company)
    dep_gen.add_level('data_center',number=2,level_type=faker.street_name)
    dep_gen.add_level('device',number=2,level_type=faker.msisdn)

    # Create meta-data
    deployment_df = dep_gen.generate_deployment()
    return deployment_df

In [69]:
def _is_deployment_exist(path):
    # Checking shared path for the devices table
    return os.path.exists(f'/v3io/bigdata/{path}')

In [70]:
def _get_deployment_from_kv(client, path):
    print(f'Retrieving deployment from {path}')
    # Read the devices table from our KV store
    deployment_df = client.read(backend='kv', table=path)
    
    # Reset index to column
    deployment_df.index.name = 'device'
    deployment_df = deployment_df.reset_index()
    return deployment_df

In [71]:
def _save_deployment_to_kv(path, df, client=v3f.Client('framesd:8081')):
    # Save deployment to our KV store
    client.write(backend='kv', table='netops_devices',dfs=df, index_cols=['device'])

In [72]:
def get_or_create_deployment(path, save_to_cloud=False, client=v3f.Client('framesd:8081')):
    if _is_deployment_exist(path):
        # Get deployment from KV
        deployment_df = _get_deployment_from_kv(client, path)
    else:
        # Create deployment
        deployment_df = _create_deployment()
        
        if save_to_cloud:
            _save_deployment_to_kv(path, deployment_df, client)

    return deployment_df

In [73]:
def save_metrics_to_tsdb(context, metrics: pd.DataFrame):
    context.v3f.write('tsdb', context.metrics_table, metrics)

In [74]:
def save_metrics_to_parquet(context, metrics: pd.DataFrame):
    metrics.to_parquet(fname=os.path.join('/', 'v3io', 'bigdata', context.metrics_table, time.strftime('%Y%m%dT%H:%M:%S')))

### Init context

In [88]:
def init_context(context):
    # Create our DB client
    client = v3f.Client('http://' + os.environ['V3IO_FRAMESD'])
    setattr(context, 'v3f', client)
    
    # Create or Get Deployment
    deployment_df = get_or_create_deployment(os.environ['DEPLOYMENT_TABLE'], os.environ['SAVE_DEPLOYMENT'], client)
    
    deployment_df['cpu_utilization'] = 70
    deployment_df['latency'] = 0
    deployment_df['packet_loss'] = 0
    deployment_df['throughput'] = 290
    deployment_df.head()
    
    # Get metrics configuration
    with open(os.getenv('METRICS_CONFIGURATION_FILEPATH', '/configurations/metrics_configuration.yaml'), 'r') as f:
        metrics_configuration = yaml.load(f)
        
    # Create metrics generator
    initial_timestamp = int(os.getenv('INITIAL_TIMESTAMP', time.time()))
    met_gen = metrics_generator.Generator_df(metrics_configuration, 
                                             user_hierarchy=deployment_df, 
                                             initial_timestamp=initial_timestamp)
    setattr(context, 'metric_generator', met_gen)
    
    # Set batch endtime
    secs_to_generate = os.getenv('SECS_TO_GENERATE', 10)
    setattr(context, 'secs_to_generate', secs_to_generate)
    
    # Get saving configuration
    save_to_tsdb = os.getenv('SAVE_TO_TSDB', True)
    setattr(context, 'save_to_tsdb', save_to_tsdb)
    
    metrics_table = os.environ['SAVE_TO']
    setattr(context, 'metrics_table', metrics_table)

    save_function = save_metrics_to_tsdb if os.getenv('SAVE_TO_TSDB', True) else save_metrics_to_parquet
    setattr(context, 'write', save_function)

### Handler

In [91]:
def handler(context, event):

    # Create metrics generator based on YAML configuration and deployment
    metrics = context.metric_generator.generate_range(start_time=datetime.datetime.now(),
                                     end_time=datetime.datetime.now()+datetime.timedelta(seconds=int(context.secs_to_generate)),
                                     as_df=True,
                                     as_iterator=True)
    
    # Save Generated metrics
    context.write(context, metrics)

## Test

In [ ]:
# nuclio: ignore
init_context(context)
event = nuclio.Event(body='')
output = handler(context, event)
output

Retrieving deployment from netops_devices


/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


# nuclio: ignore
event = nuclio.Event

In [ ]:
%nuclio deploy -p netops -n generator -c